In [1]:
from UCLSE.environment import Market_session, yamlLoad
from UCLSE.test.utils import identical_replay_vars,side_by_side_period_by_period_difference_checker
from UCLSE.exchange import Exchange
import copy

I have extensively rewritten the exchange.py file to account for variable quantity orders. The main complication of this that any new order can cause a greater number of outcomes. Consider a new bid. It can have the following effects:

- 0 Add itself to the LOB and do nothing
- 1 Bid exactly fills a best bid ask, two orders are deleted off LOB and two traders informed.
- 2a Bid completely fills with any number of ask orders at any number of prices. Any number of orders are deleted and any number of traders are informed.
- 2a As above, but only partially fill an existing ask on the LOB. Ask order owner needs to be made aware of order change.
- 3 Bid Partially filled - As above but bid owner needs to be made aware of change. 
<br>
There are also bookkeeping considerations which I haven't considered yet. 

But, to check the ammendments haven't messed up the original calculations, we should run parallel experiments using the old and new exchange methods to ensure that they result in the same thing. 

In [2]:
import os
pa=os.getcwd()
config_name='config\\mkt_cfg.yml'
config_path=os.path.join(pa,config_name)
environ_dic=yamlLoad(config_path)

#just updates the tradefile every time there is a trade
environ_dic['dump_each_trade']=True
environ_dic['verbose']=False
environ_dic['end_time']=50

I have add process_order as an attribute of the session object, to allow selection of which process order mechanism we use. It will default to the old one.

First check that the new method works. Begin by instantiating a Market_session and then copying it exactly.

In [3]:
import copy
sess=Market_session(**environ_dic)
sess.process_order=sess.exchange.process_order3w
sess1=copy.deepcopy(sess)


Then we can run the first simulation, store the events and push those events into the second simulation session to make sure that the end result is the same.

In [4]:
sess.simulate(sess.trade_stats_df3,recording=True)
sess1.simulate(sess1.trade_stats_df3,replay_vars=sess.replay_vars,recording=True)
(sess1.df.fillna(0)==sess.df.fillna(0)).all() #note that the best_bid or best_bid do not always give a True value. Because nan!=nan.

Ask  leg 0  lifts best  Bid 95
counterparty B23 price 105
[{'time': 2.3875, 'price': 105, 'party1': 'B23', 'qty': 1, 'party2': 'S23', 'p2_qid': 4.0, 'type': 'Trade', 'p1_qid': 2}]
Ask  leg 0  lifts best  Bid 105
counterparty B11 price 105
[{'time': 5.237500000000041, 'price': 105, 'party1': 'B11', 'qty': 1, 'party2': 'S18', 'p2_qid': 20.0, 'type': 'Trade', 'p1_qid': 13}]
Ask  leg 0  lifts best  Bid 95
counterparty B24 price 105
[{'time': 6.087500000000053, 'price': 105, 'party1': 'B24', 'qty': 1, 'party2': 'S37', 'p2_qid': 28.0, 'type': 'Trade', 'p1_qid': 25}]
Ask  leg 0  lifts best  Bid 102
counterparty B02 price 105
[{'time': 7.875000000000078, 'price': 105, 'party1': 'B02', 'qty': 1, 'party2': 'S24', 'p2_qid': 40.0, 'type': 'Trade', 'p1_qid': 38}]
Bid  leg 0  lifts best  Ask 106
counterparty S05 price 104
[{'time': 9.599999999999989, 'price': 104, 'party1': 'S05', 'qty': 1, 'party2': 'B12', 'p2_qid': 56.0, 'type': 'Trade', 'p1_qid': 55}]
Bid  leg 0  lifts best  Ask 106
counterparty 

Ask  leg 0  lifts best  Bid 105
counterparty B19 price 105
[{'time': 18.987499999999457, 'price': 105, 'party1': 'B19', 'qty': 1, 'party2': 'S07', 'p2_qid': 227.0, 'type': 'Trade', 'p1_qid': 216}]
Bid  leg 0  lifts best  Ask 106
counterparty S15 price 106
[{'time': 19.362499999999436, 'price': 106, 'party1': 'S15', 'qty': 1, 'party2': 'B31', 'p2_qid': 232.0, 'type': 'Trade', 'p1_qid': 230}]
Ask  leg 0  lifts best  Bid 105
counterparty B26 price 105
[{'time': 20.649999999999363, 'price': 105, 'party1': 'B26', 'qty': 1, 'party2': 'S32', 'p2_qid': 255.0, 'type': 'Trade', 'p1_qid': 250}]
Ask  leg 0  lifts best  Bid 96
counterparty B17 price 105
[{'time': 21.54999999999931, 'price': 105, 'party1': 'B17', 'qty': 1, 'party2': 'S27', 'p2_qid': 271.0, 'type': 'Trade', 'p1_qid': 266}]
Ask  leg 0  lifts best  Bid 96
counterparty B20 price 105
[{'time': 22.037499999999284, 'price': 105, 'party1': 'B20', 'qty': 1, 'party2': 'S19', 'p2_qid': 283.0, 'type': 'Trade', 'p1_qid': 275}]
Ask  leg 0  lifts 

expid                    True
time                     True
GVWY      balance_sum    True
          n              True
          pc             True
ZIC       balance_sum    True
          n              True
          pc             True
ZIP       balance_sum    True
          n              True
          pc             True
SHVR      balance_sum    True
          n              True
          pc             True
best_bid                 True
best_ask                 True
dtype: bool

This function checks that the two replay_vars attributes are the same and returns the difference if it exists:
    

In [5]:
identical_replay_vars(sess,sess1)

no differences found


{}

In [6]:
sess.exchange.tape

[{'oid': -64,
  'order qty': 1,
  'order_issue_time': 0.8125549527697913,
  'p1_qid': 2,
  'p2_qid': 4.0,
  'party1': 'B23',
  'party2': 'S23',
  'price': 105,
  'profit': 10,
  'qty': 1,
  'status': 'full',
  'time': 2.3875,
  'type': 'Trade'},
 {'oid': -59,
  'order qty': 1,
  'order_issue_time': 3.400670975398472,
  'p1_qid': 13,
  'p2_qid': 20.0,
  'party1': 'B11',
  'party2': 'S18',
  'price': 105,
  'profit': 10,
  'qty': 1,
  'status': 'full',
  'time': 5.237500000000041,
  'type': 'Trade'},
 {'oid': -78,
  'order qty': 1,
  'order_issue_time': 5.804099240983145,
  'p1_qid': 25,
  'p2_qid': 28.0,
  'party1': 'B24',
  'party2': 'S37',
  'price': 105,
  'profit': 10,
  'qty': 1,
  'status': 'full',
  'time': 6.087500000000053,
  'type': 'Trade'},
 {'oid': -65,
  'order qty': 1,
  'order_issue_time': 7.8573640751744485,
  'p1_qid': 38,
  'p2_qid': 40.0,
  'party1': 'B02',
  'party2': 'S24',
  'price': 105,
  'profit': 9,
  'qty': 1,
  'status': 'full',
  'time': 7.875000000000078,


For a more forensic approach to difference checking, this function will look for differences at the end of every period and stop if one occurs.

In [7]:
sess=Market_session(**environ_dic)
sess.process_order=sess.exchange.process_order3w
sess1=copy.deepcopy(sess)
side_by_side_period_by_period_difference_checker(sess,sess1)


Ask  leg 0  lifts best  Bid 105
counterparty B12 price 105
[{'time': 7.500000000000073, 'price': 105, 'party1': 'B12', 'qty': 1, 'party2': 'S17', 'p2_qid': 81.0, 'type': 'Trade', 'p1_qid': 67}]
Ask  leg 0  lifts best  Bid 105
counterparty B12 price 105
[{'time': 7.500000000000073, 'price': 105, 'party1': 'B12', 'qty': 1, 'party2': 'S17', 'p2_qid': 81.0, 'type': 'Trade', 'p1_qid': 67}]
Ask  leg 0  lifts best  Bid 105
counterparty B15 price 105
[{'time': 8.137500000000072, 'price': 105, 'party1': 'B15', 'qty': 1, 'party2': 'S02', 'p2_qid': 96.0, 'type': 'Trade', 'p1_qid': 90}]
Ask  leg 0  lifts best  Bid 105
counterparty B15 price 105
[{'time': 8.137500000000072, 'price': 105, 'party1': 'B15', 'qty': 1, 'party2': 'S02', 'p2_qid': 96.0, 'type': 'Trade', 'p1_qid': 90}]
Ask  leg 0  lifts best  Bid 105
counterparty B05 price 105
[{'time': 8.937500000000027, 'price': 105, 'party1': 'B05', 'qty': 1, 'party2': 'S20', 'p2_qid': 106.0, 'type': 'Trade', 'p1_qid': 105}]
Ask  leg 0  lifts best  Bid 

Bid  leg 0  lifts best  Ask 104
counterparty S34 price 104
[{'time': 34.38749999999926, 'price': 104, 'party1': 'S34', 'qty': 1, 'party2': 'B29', 'p2_qid': 690.0, 'type': 'Trade', 'p1_qid': 624}]
Bid  leg 0  lifts best  Ask 104
counterparty S34 price 104
[{'time': 34.38749999999926, 'price': 104, 'party1': 'S34', 'qty': 1, 'party2': 'B29', 'p2_qid': 690.0, 'type': 'Trade', 'p1_qid': 624}]
Ask  leg 0  lifts best  Bid 98
counterparty B14 price 104
[{'time': 34.599999999999305, 'price': 104, 'party1': 'B14', 'qty': 1, 'party2': 'S39', 'p2_qid': 694.0, 'type': 'Trade', 'p1_qid': 692}]
Ask  leg 0  lifts best  Bid 98
counterparty B14 price 104
[{'time': 34.599999999999305, 'price': 104, 'party1': 'B14', 'qty': 1, 'party2': 'S39', 'p2_qid': 694.0, 'type': 'Trade', 'p1_qid': 692}]
Bid  leg 0  lifts best  Ask 105
counterparty S36 price 105
[{'time': 36.24999999999968, 'price': 105, 'party1': 'S36', 'qty': 1, 'party2': 'B12', 'p2_qid': 730.0, 'type': 'Trade', 'p1_qid': 726}]
Bid  leg 0  lifts be

Now make one session object use the process_order and one the new process_order

In [8]:
sess=Market_session(**environ_dic)

sess1=copy.deepcopy(sess)
sess.process_order=sess.exchange.process_order3w
side_by_side_period_by_period_difference_checker(sess,sess1)

Ask  leg 0  lifts best  Bid 95
counterparty B16 price 105
[{'time': 2.5000000000000018, 'price': 105, 'party1': 'B16', 'qty': 1, 'party2': 'S19', 'p2_qid': 7.0, 'type': 'Trade', 'p1_qid': 2}]
[{'time': 2.5000000000000018, 'price': 105, 'party1': 'B16', 'qty': 1, 'party2': 'S19', 'p2_qid': 7, 'type': 'Trade', 'p1_qid': 2}]
Ask  leg 0  lifts best  Bid 95
counterparty B02 price 105
[{'time': 6.025000000000052, 'price': 105, 'party1': 'B02', 'qty': 1, 'party2': 'S04', 'p2_qid': 64.0, 'type': 'Trade', 'p1_qid': 41}]
[{'time': 6.025000000000052, 'price': 105, 'party1': 'B02', 'qty': 1, 'party2': 'S04', 'p2_qid': 64, 'type': 'Trade', 'p1_qid': 41}]
Ask  leg 0  lifts best  Bid 105
counterparty B32 price 105
[{'time': 7.950000000000079, 'price': 105, 'party1': 'B32', 'qty': 1, 'party2': 'S07', 'p2_qid': 109.0, 'type': 'Trade', 'p1_qid': 74}]
[{'time': 7.950000000000079, 'price': 105, 'party1': 'B32', 'qty': 1, 'party2': 'S07', 'p2_qid': 109, 'type': 'Trade', 'p1_qid': 74}]
Ask  leg 0  lifts bes

Ask  leg 0  lifts best  Bid 97
counterparty B06 price 106
[{'time': 35.349999999999476, 'price': 106, 'party1': 'B06', 'qty': 1, 'party2': 'S08', 'p2_qid': 826.0, 'type': 'Trade', 'p1_qid': 818}]
[{'time': 35.349999999999476, 'price': 106, 'party1': 'B06', 'qty': 1, 'party2': 'S08', 'p2_qid': 826, 'type': 'Trade', 'p1_qid': 818}]
Ask  leg 0  lifts best  Bid 97
counterparty B07 price 106
[{'time': 35.699999999999555, 'price': 106, 'party1': 'B07', 'qty': 1, 'party2': 'S24', 'p2_qid': 840.0, 'type': 'Trade', 'p1_qid': 827}]
[{'time': 35.699999999999555, 'price': 106, 'party1': 'B07', 'qty': 1, 'party2': 'S24', 'p2_qid': 840, 'type': 'Trade', 'p1_qid': 827}]
Ask  leg 0  lifts best  Bid 98
counterparty B27 price 105
[{'time': 35.79999999999958, 'price': 105, 'party1': 'B27', 'qty': 1, 'party2': 'S17', 'p2_qid': 842.0, 'type': 'Trade', 'p1_qid': 810}]
[{'time': 35.79999999999958, 'price': 105, 'party1': 'B27', 'qty': 1, 'party2': 'S17', 'p2_qid': 842, 'type': 'Trade', 'p1_qid': 810}]
Ask  l

In [19]:
identical_replay_vars(sess,sess1)

no differences found


{}

And for completion, swap it over so the new process_order works in replay mode:

In [9]:
sess=Market_session(**environ_dic)
sess.verbose=True
sess1=copy.deepcopy(sess)
sess1.process_order=sess1.exchange.process_order3w
side_by_side_period_by_period_difference_checker(sess,sess1)


trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial

replay True SHVR   0.0 []
Trader Quote: [S05 Ask P=095 Q=1 T= 5.36 QID:None OID:-46]
Trader Quote: [S05 Ask P=095 Q=1 T= 7.03 QID:44 OID:-46]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S22 Ask P=095 Q=1 T= 2.95 QID:None OID:-63]
Trader Quote: [S22 Ask P=648 Q=1 T= 7.08 QID:45 OID:-63]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S22 Ask P=095 Q=1 T= 2.95 QID:None OID:-63]
Trader Quote: [S22 Ask P=648 Q=1 T= 7.08 QID:45 OID:-63]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
r

replay True ZIC   0.0 []
Trader Quote: [S22 Ask P=095 Q=1 T= 2.95 QID:None OID:-63]
Trader Quote: [S22 Ask P=793 Q=1 T=11.06 QID:98 OID:-63]

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S25 Ask P=095 Q=1 T= 7.31 QID:None OID:-66]
Trader Quote: [S25 Ask P=403 Q=1 T=11.07 QID:99 OID:-66]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S25 Ask P=095 Q=1 T= 7.31 QID:None OID:-66]
Trader Quote: [S25 Ask P=403 Q=1 T=11.07 QID:99 OID:-66]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

t

trial0001;  
replay True ZIP   0.0 []
Trader Quote: [B16 Bid P=105 Q=1 T= 6.26 QID:None OID:-17]
Trader Quote: [B16 Bid P=093 Q=1 T=17.40 QID:173 OID:-17]

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [B31 Bid P=106 Q=1 T=17.23 QID:None OID:-32]
Trader Quote: [B31 Bid P=053 Q=1 T=17.41 QID:174 OID:-32]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [B31 Bid P=106 Q=1 T=17.23 QID:None OID:-32]
Trader Quote: [B31 Bid P=053 Q=1 T=17.41 QID:174 OID:-32]

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [B23 Bid P=106 Q=1 T=16.52 QID:None OID:-24]
Trader Quote: [B23 Bid P=066 Q=1 T=17.42 QID:175 OID:-24]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [B23 Bid P=106 Q=1 T=16.52 QID:None OID:-24]
Trader Quote: [B23 Bid P=066 Q=1 T=17.42 QID:175 OID:-24]

trial0001;  

trial0001;  

trial0001;  
replay False ZIP   0.0 []
Trader Quote: [B24 Bid P=106 Q=1 T=17.13 QID:None OID:-25]
Trader Quote: [B24 Bid P=071 Q=1 T=17.45 QID:176 OID:-25]

trial0001;  
replay True ZIP   0.

replay True ZIC   0.0 []
Trader Quote: [S22 Ask P=095 Q=1 T= 2.95 QID:None OID:-63]
Trader Quote: [S22 Ask P=295 Q=1 T=21.99 QID:257 OID:-63]

trial0001;  

trial0001;  

trial0001;  
replay False ZIP   0.0 []
Trader Quote: [B16 Bid P=105 Q=1 T= 6.26 QID:None OID:-17]
Trader Quote: [B16 Bid P=093 Q=1 T=22.01 QID:258 OID:-17]

trial0001;  
replay True ZIP   0.0 []
Trader Quote: [B16 Bid P=105 Q=1 T= 6.26 QID:None OID:-17]
Trader Quote: [B16 Bid P=093 Q=1 T=22.01 QID:258 OID:-17]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIP   0.0 []
Trader Quote: [B19 Bid P=106 Q=1 T=17.17 QID:None OID:-20]
Trader Quote: [B19 Bid P=090 Q=1 T=22.05 QID:259 OID:-20]

trial0001;  
replay True ZIP   0.0 []
Trader Quote: [B19 Bid P=106 Q=1 T=17.17 QID:None OID:-20]
Trader Quote: [B19 Bid P=090 Q=1 T=22.05 QID:259 OID:-20]

trial0001;  
replay False ZIP   0.0 []
Trader Quote: [B19 Bid P=106 Q=1 T=17.17 QID:None OID:-20]
Trader Quote: [B19 Bid P=090 Q=1 T=22.06 QID:260 

...


trial0001;  
replay False ZIP   0.0 []
Trader Quote: [S24 Ask P=097 Q=1 T=29.43 QID:None OID:-65]
Trader Quote: [S24 Ask P=105 Q=1 T=33.99 QID:550 OID:-65]
[{'time': 33.987499999999166, 'price': 105, 'party1': 'B19', 'qty': 1, 'party2': 'S24', 'p2_qid': 550, 'type': 'Trade', 'p1_qid': 535}]

trial0001;  
replay True ZIP   0.0 []
Trader Quote: [S24 Ask P=097 Q=1 T=29.43 QID:None OID:-65]
Trader Quote: [S24 Ask P=105 Q=1 T=33.99 QID:550 OID:-65]
Ask  leg 0  lifts best  Bid 105
counterparty B19 price 105
[{'time': 33.987499999999166, 'price': 105, 'party1': 'B19', 'qty': 1, 'party2': 'S24', 'p2_qid': 550.0, 'type': 'Trade', 'p1_qid': 535}]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIP   0.0 []
Trader Quote: [S23 Ask P=098 Q=1 T=33.66 QID:None OID:-64]
Trader Quote: [S23 Ask P=108 Q=1 T=34.04 QID:551 OID:-64]

trial0001;  
replay True ZIP   0.0 []
Trader Quote: [S23 Ask P=098 Q=1 T=33.66 QID:None OID:-64]
Trader Quote: [

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S19 Ask P=096 Q=1 T=20.05 QID:None OID:-60]
Trader Quote: [S19 Ask P=914 Q=1 T=37.41 QID:645 OID:-60]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S19 Ask P=096 Q=1 T=20.05 QID:None OID:-60]
Trader Quote: [S19 Ask P=914 Q=1 T=37.41 QID:645 OID:-60]

trial0001;  

trial0001;  

trial0001;  
replay False ZIP   0.0 []
Trader Quote: [B04 Bid P=107 Q=1 T=25.87 QID:None OID:-5]
Trader Quote: [B04 Bid P=103 Q=1 T=37.44 QID:646 OID:-5]

trial0001;  
replay True ZIP   0.0 []
Trader Quote: [B04 Bid P=107 Q=1 T=25.87 QID:None OID:-5]
Trader Quote: [B04 Bid P=103 Q=1 T=37.44 QID:646 OID:-5]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [B03 Bid P=107 Q=1 T=33.25 QID:None OID:-4]
Trader Quote: [B03 Bid P=080 Q=1 T=37.47 QID:647 OID:-4]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [B03 Bid P=107 Q=1 T=33.25 QID:

replay True ZIC   0.0 []
Trader Quote: [S27 Ask P=097 Q=1 T=27.52 QID:None OID:-68]
Trader Quote: [S27 Ask P=532 Q=1 T=41.05 QID:741 OID:-68]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [B31 Bid P=107 Q=1 T=30.15 QID:None OID:-32]
Trader Quote: [B31 Bid P=003 Q=1 T=41.09 QID:742 OID:-32]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [B31 Bid P=107 Q=1 T=30.15 QID:None OID:-32]
Trader Quote: [B31 Bid P=003 Q=1 T=41.09 QID:742 OID:-32]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S11 Ask P=096 Q=1 T= 9.61 QID:None OID:-52]
Trader Quote: [S11 Ask P=630 Q=1 T=41.19 QID:743 OID:-52]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S11 Ask P=096 Q=1 T= 9.61 QID:None OID:-52]
Trader Quote: [S11 Ask P=630 Q=1 T=41.


trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S25 Ask P=098 Q=1 T=38.90 QID:None OID:-66]
Trader Quote: [S25 Ask P=527 Q=1 T=46.10 QID:827 OID:-66]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S25 Ask P=098 Q=1 T=38.90 QID:None OID:-66]
Trader Quote: [S25 Ask P=527 Q=1 T=46.10 QID:827 OID:-66]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S11 Ask P=096 Q=1 T= 9.61 QID:None OID:-52]
Trader Quote: [S11 Ask P=977 Q=1 T=46.23 QID:828 OID:-52]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S11 Ask P=096 Q=1 T= 9.61 QID:None OID:-52]
Trader Quote: [S11 Ask P=977 Q=1 T=46.23 QID:828 OID:-52]

trial0001


trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [B31 Bid P=107 Q=1 T=30.15 QID:None OID:-32]
Trader Quote: [B31 Bid P=096 Q=1 T=49.63 QID:884 OID:-32]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [B31 Bid P=107 Q=1 T=30.15 QID:None OID:-32]
Trader Quote: [B31 Bid P=096 Q=1 T=49.63 QID:884 OID:-32]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False GVWY   8.0 [{'order qty': 1, 'profit': 8, 'qty': 1, 'time': 28.012499999998944, 'status': 'full', 'price': 99, 'party1': 'S31', 'party2': 'B08', 'p2_qid': 379, 'p1_qid': 367, 'type': 'Trade', 'order_issue_time': 27.06541899540877, 'oid': -9}]
Trader Quote: [B08 Bid P=109 Q=1 T=48.23 QID:None OID:-9]
Trader Quote: [B08 Bid P=109 Q=1 T=49.66 QID:885 OID:-9]

trial0001;  
replay True GVWY   8.0 [{'order qty': 1, 'profit': 8, 'qty': 1, 'time': 28.012499999998944, 'status': 'full', 'price': 99, 'party1': 'S31', 'party2': 'B08', 'p2_qid': 379.0, 'p1_qid': 367, 'type': 'Trade'